In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import math
from math import sqrt
from scipy.integrate import quad

def delta_calculator(S, K, r, tau, sigma):
    if tau > 0:
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
        return norm.cdf(d1)
    else:
        return 1 if S >= K else 0

def rebalancing_days(N, num_steps):
    return [round(i * N / num_steps) for i in range(1, num_steps + 1)]
def portfolio_calculator(real_path_1,real_path_2,K,r,T, sigma_1,sigma_2, num_steps,N, alpha,k_1,k_2):
    S0_1 = real_path_1[0]
    S0_2 = real_path_2[0]
    sigma = sqrt((alpha ** 2) * (sigma_1 ** 2) + ((1 - alpha) ** 2) * (sigma_2 ** 2) + 2 * rho * alpha * (1 - alpha) * sigma_1 * sigma_2)
    dt = T / N
    reb_days = rebalancing_days(N, num_steps)
    initial_portfolio_price = alpha * S0_1 + (1 - alpha) * S0_2

    delta_initial_1 = delta_calculator(S0_1, K, r, T, sigma_1)
    delta_initial_2 = delta_calculator(S0_2, K, r, T, sigma_2)

    portfolio_value_1_1 = S0_1 * delta_initial_1 * (1 - k_1)
    portfolio_value_2_2 = S0_2 * delta_initial_2 * (1 - k_2)

    delta_before_1 = delta_initial_1
    delta_before_2 = delta_initial_2

    for t in range(1, N + 1):
        St_1 = real_path_1[t]
        St_2 = real_path_2[t]
        tau = T - t * dt

        if t in reb_days and tau > 0:
            delta_1 = delta_calculator(St_1, K, r, tau, sigma_1)
            delta_2 = delta_calculator(St_2, K, r, tau, sigma_2)

            portfolio_value_1_1 = (1 + r * dt) * portfolio_value_1_1 + (delta_1 - delta_before_1) * St_1 * (
                        1 - np.sign(delta_1 - delta_before_1) * k_1)
            portfolio_value_2_2 = (1 + r * dt) * portfolio_value_2_2 + (delta_2 - delta_before_2) * St_2 * (
                        1 - np.sign(delta_2 - delta_before_2) * k_2)

            delta_before_1 = delta_1
            delta_before_2 = delta_2

        if  t in reb_days and tau == 0:
            S_T = alpha * St_1 + (1 - alpha) * St_2

            delta_final_1 = delta_calculator(St_1, K, r, 0, sigma_1)
            delta_final_2 = delta_calculator(St_2, K, r, 0, sigma_2)


            final_portfolio_value_1_1 = (1 + r * dt) * portfolio_value_1_1 + (delta_final_1 - delta_before_1) * St_1 * (
                        1 - np.sign(delta_final_1 - delta_before_1) * k_1) - delta_final_1 * St_1 + max(S_T - K, 0)
            simulatedprice_1_1 = np.exp(-r * T) * final_portfolio_value_1_1


            final_portfolio_value_2_2 = (1 + r * dt) * portfolio_value_2_2 + (delta_final_2 - delta_before_2) * St_2 * (
                        1 - np.sign(delta_final_2 - delta_before_2) * k_2) - delta_final_2 * St_2 + max(S_T - K, 0)
            simulatedprice_2_2 = np.exp(-r * T) * final_portfolio_value_2_2

            Measure = np.exp(-r * T) * max(S_T - K, 0)
            
            return simulatedprice_1_1, simulatedprice_2_2, Measure

df = pd.read_csv("combined_data_JSE_x@@dec.csv")
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date")

asset1 = df["Close_file1"].values
asset2 = df["Close_file9"].values

estimation_length = 252
hedging_length = 252
total_length = estimation_length + hedging_length + 1

r = 0.09
T = 1
N = hedging_length
k = 0.0

k_1_values = [0.001]
k_2_values = [0.01]
num_steps_list = [2, 4, 12, 52, 252]
alpha_values = np.round(np.linspace(0, 0, 1), 1)

rho = 0.89

results = []    
for start in range(len(asset1) - total_length + 1):
    window_est_1 = asset1[start: start + estimation_length]
    window_est_2 = asset2[start: start + estimation_length]
    window_hedge_1 = asset1[start + estimation_length: start + total_length]
    window_hedge_2 = asset2[start + estimation_length: start + total_length]

    returns_1 = np.diff(np.log(window_est_1))
    returns_2 = np.diff(np.log(window_est_2))
    dt = 1 / 252
    sigma_1 = np.std(returns_1, ddof=1) / sqrt(dt)
    sigma_2 = np.std(returns_2, ddof=1) / sqrt(dt)
    rho = np.corrcoef(returns_1, returns_2)[0, 1]

    S0_1 = window_est_1[-1]
    S0_2 = window_est_2[-1]
    
    K = S0_2 

    for alpha in alpha_values:
        initial_portfolio_price_St = (alpha * S0_1 + (1 - alpha) * S0_2)

        for k_1 in k_1_values:
            k_1 = float(k_1)
            for k_2 in k_2_values:
                k_2 = float(k_2)
                for num_steps in num_steps_list:
                    num_steps = int(num_steps)

                    simulatedprice_1_1, simulatedprice_2_2, Measure = portfolio_calculator(window_hedge_1, window_hedge_2,
                        K, r, T, sigma_1, sigma_2,num_steps, N,alpha,k_1, k_2)

                    results.append({'start_index': start,'alpha': alpha,'num_steps': num_steps,'k_1': k_1,'k_2': k_2,
                        'simulatedprice_1_1': simulatedprice_1_1,'simulatedprice_2_2': simulatedprice_2_2,'measure': Measure
                    })

df_results = pd.DataFrame(results)

final_results = []


for (alpha, num_steps, k_1, k_2), group in df_results.groupby(['alpha','num_steps', 'k_1', 'k_2']):
    simulatedprices_1_1 = group['simulatedprice_1_1'].tolist()
    simulatedprices_2_2 = group['simulatedprice_2_2'].tolist()
    measures = group['measure'].tolist()

    def mean_std(vals):
        mean_val = sum(vals) / len(vals)
        variance = sum([(x - mean_val) ** 2 for x in vals]) / len(vals)
        std_val = math.sqrt(variance)
        return mean_val, std_val

    avg_1_1, std_1_1 = mean_std(simulatedprices_1_1)
    avg_2_2, std_2_2 = mean_std(simulatedprices_2_2)
    avg_measure, std_measure = mean_std(measures)
    final_results.append((alpha, num_steps, k_1, k_2, avg_1_1, std_1_1,
                          avg_2_2, std_2_2, avg_measure, std_measure)) 
    
df_final = pd.DataFrame(final_results, columns=[
    'alpha', 'num_steps', 'k_1', 'k_2','avg_simulatedprice_1_1', 'std_simulatedprice_1_1',
    'avg_simulatedprice_2_2', 'std_simulatedprice_2_2','avg_measure', 'std_measure'])


df_final.to_csv('hedging_results_corrected_high_correlation_x5@dec.csv', index=False)

print("Corrected full results saved to 'hedging_results_corrected_high_correlation_x5@dec.csv'")